# A Malthus economy with technological growth

**Time:** Discrete and represented by $ t \in {0,1,...,\infty} $

**Production function:** Cobb-Douglas
$$ Y_t = L_t^{1-\alpha}(AX)^\alpha $$
$A$ and $X$ are independent of time. $A$ is the technology-level and $X$ is the amount of land available.

The production function can also be written as production pr. capita, $y_t$, which is found by dividing the equation above by $L_t$:
$$ y_t = \frac{Y_t}{L_t} = (\frac{AX}{L_t})^\alpha $$

**Fertility curve:**
$$n_t = \frac{1-\beta}{\lambda}y_t (1-\tau)$$

In the equation for the fertility-curve the expression $1-\beta$ changes with preferences/culture about having children, $\lambda$ represents the lower costs of having children and $\tau$ represents taxes in society.


**Transition-equation:**
$$L_{t+1} = n_t L_t + (1-\mu) L_t$$
In the transition equation above $L_0$ is given.

A key assumption in the Malthus economy is that the birth rate, $n_t$, rises with income pr. capita, $y_t$, such that in steady state the birth rate is going to be: $$ n^* = \eta * y^* $$ $\eta$ is a positive parameter.

## Analytical solution

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details.

In [1]:
import ModelClass
import sympy as sm
from IPython.display import display, Markdown

model = ModelClass.MalthusModel()

transition_eq = model.symbolic_L()

latex_with_code = r"$ \Large{ L_{t+1} = " + sm.latex(transition_eq) + r"}$"

print("To derieve the analytical steady state the transition equation describing the change in labor force will be used:")
display(Markdown(latex_with_code))

To derieve the analytical steady state the transition equation describing the change in labor force will be used:


$ \Large{ L_{t+1} = L_{t} \left(1 - \mu\right) + L_{t}^{1 - \alpha} \eta \left(A X\right)^{\alpha}}$

In [7]:
# Steady state expression for labor force
L_ss_symbolic = model.symbolic_ss_L()

latex_with_code = r"$ \Large{ L^{*} = " + sm.latex(L_ss_symbolic) + r"}$"

print("Steady state expression for the labor force:")
display(Markdown(latex_with_code))


# Steady state expression for output pr. worker
y_ss_symbolic = model.symbolic_ss_y()

latex_with_code = r"$ \Large{ y^{*} = " + sm.latex(y_ss_symbolic) + r"}$"

print("Steady state expression for output pr. worker:")
display(Markdown(latex_with_code))


Steady state expression for the labor force:


$ \Large{ L^{*} = \left(\frac{\eta \left(A X\right)^{\alpha}}{\mu}\right)^{\frac{1}{\alpha}}}$

Steady state expression for output pr. worker:


$ \Large{ y^{*} = \left(A X \left(\frac{\eta \left(A X\right)^{\alpha}}{\mu}\right)^{- \frac{1}{\alpha}}\right)^{\alpha}}$

It's possible to simplify the expression for $L^*$ and $y^*$ a lot, something which symbolic python is not great at.

## Numerical solution to the Malthus model

To begin with we find the amount of labor force, $L^*$, in the steady state.

In the code below we have used **optimize.root** in the scipy to find the optimal value of $L_t$ where the difference between $L_t$ and $L_{t+1}$ is equal to zero. In that scenario we have found the steady state value of $L^*$.

In the **ModelClass.py** the parameters for the model have been defined, as well as the functions for the model.

The model doesn't have any problem with convergence, but that is not surprising since it doesn't have multiple local minima which could "confuse" the optimization algorithm. However the optimization function **numerical_solution_steady_state** in the ModelClass has been implemented with **multi start**, so it should also be possible to find the global minima in case anyone wants to update the model where the optimization could be harder for the algorithm i.e. the case where there are multiple local minima.

In [16]:
model = ModelClass.MalthusModel()

labor_ss, output_ss, output_pr_worker_ss, birth_rate_ss, residual_ss = model.numerical_solution_steady_state()

print("Steady state value for labor force:", format(labor_ss[0], ".10f"))
print("Steady state value for output:", format(output_ss[0], ".10f"))
print("Steady state value for output pr. worker:", format(output_pr_worker_ss[0], ".10f"))
print("Steady state value for birth rate:", format(birth_rate_ss[0], ".10f"))
print("State state residual from the optimization algorithm:", format(residual_ss[0], ".10f"))

Steady state value for labor force: 2.5600000000
Steady state value for output: 1.6000000000
Steady state value for output pr. worker: 0.6250000000
Steady state value for birth rate: 0.5000000000
State state residual from the optimization algorithm: 0.0000000000


# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model.

**Extending the model with technological progress.**

# Conclusion

Add concise conclusion. 

# Ideas for extensions
- Change of the production function to an i.e. CES production function or maybe another production function also.
- Extension of the model with growth in technology


